# Stock_Query
This script runs the output of the Query_List.ipynb (Query_List.csv) and returns 20 years of daily-time series data for each stock. Data is pulled from the Alpha Vantage API. Data is then to be used in analytica for research.

Script PFD:

1. FortuneList Sorting 
2. Ticker_Lookup(in FortuneList) 
3. Symbol_Corrections(in FortuneList) 
4. Stock_Query 
5. StockMarketCrash_Analytica

In [1]:
import pandas as pd
query_list = pd.read_csv('../Resources/S&P500_List.csv', encoding='ISO 8859-1')

query_list = query_list.rename(columns={
    'Symbol' : 'ticker',
    'GICS\xa0Sector':'Sector'
})

query_list.head()
tickerlist = query_list['ticker'].tolist()
tickerlist

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'ATO',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF.B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',

In [ ]:
'''
SPAM API

The API is spammed until the call limit response is returned. The Dataset is then saved off and the API is spammed again
until a successful call return is hit.

'''


import time
import pandas as pd
from config import api_key_av
import requests

# creating limit response note from AV
bad_call = 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'

# creating loop counters & summary stat counters
success = 0 #success API call counter
failed = 0 #failed API call counter

now = int(time.time())
suffix = '1577635349'

# reading in query_list for required tickers
tickers = tickerlist

# creating master dataframe (to append to) and setting save path
save_path = f'../Resources/Master_Datasets/master_dataset_{suffix}.csv'

if suffix == now:
    master_df = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker'])
    master_df.to_csv(save_path, index=False)


# terminal print out of script performance
print(f'''
!Query Initiated!
-----------------
''')

# opening dataset to append to / read pre-appended items from
master_df = pd.read_csv(save_path)
ds_list = master_df['Ticker'].unique().tolist()


# primary loop for ticker call list to API
for record, ticker in enumerate(tickers):
    
    # by-passing previously called tickers
    if ticker not in ds_list:
        
        # setting API call parameters (ticker)
        params = {
            'function' : 'TIME_SERIES_DAILY',
            'symbol' : ticker,
            'outputsize' : 'full',
            'datatype' : 'json',
            'apikey' : api_key_av
        }
        
        # base API call URL
        url = 'https://www.alphavantage.co/query?'
        
        # t-e for handeling failed API calls
        try: 
            
            response = requests.get(url, params).json()
            
            # t-e for handeling call limit
            try:
                
                # checking if call limit reached
                if response['Note'] == bad_call:
                    
                    # dataframe save & wait timer for next machine minute
                    master_df.to_csv(save_path, index=False)
                    wait_time = time.asctime(time.localtime())
                    now = int(time.time())
                    print(f'master_df saved, entering wait @ {wait_time} on item: {ticker}')
                    
                    while response['Note'] == bad_call:
                        response = requests.get(url, params).json()
                        
                    wait_time = time.asctime(time.localtime())
                    print(f'Exiting wait @ {wait_time}')
            
            except:
                
                # THE MEAT AND POTATOS
                # creating main ticker dataframe
                ticker_df = pd.DataFrame(response['Time Series (Daily)']).transpose().reset_index()

                # renaming columns to better indicators
                ticker_df = ticker_df.rename(columns={
                    'index':'Date',
                    '1. open':'Open',
                    '2. high':'High',
                    '3. low':'Low',
                    '4. close':'Close',
                    '5. volume':'Volume'
                })
                
                # pulling ticker symbol for confirmation purposes from API
                ticker_df['Ticker'] = response['Meta Data']['2. Symbol']
                
                
                # appending items to master dataframe
                master_df = master_df.append(ticker_df)
                
                row_count = ticker_df['Date'].count()
                
                print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : SUCCESS({row_count} rows)')
                
                success += 1
                
        # handeling failed API calls
        except:
            print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : FAILED')
            
            failed += 1
            
    else:
        print(f'Record {record + 1} out of {len(tickers)} Skipped. {ticker}')
            
            

# writing master dataframe to csv   
master_df.to_csv(save_path, index=False)


# terminal print out of script performance
print(f'''
DONE
------------------
Query Summary
Success: {success} 
Failed: {failed}
Total: {record + 1}
''')


!Query Initiated!
-----------------

Record 1 out of 505 Skipped. MMM
Record 2 out of 505 Skipped. ABT
Record 3 out of 505 Skipped. ABBV
Record 4 out of 505 Skipped. ABMD
Record 5 out of 505 Skipped. ACN
Record 6 out of 505 Skipped. ATVI
Record 7 out of 505 Skipped. ADBE
Record 8 out of 505 Skipped. AMD
Record 9 out of 505 Skipped. AAP
Record 10 out of 505 Skipped. AES
Record 11 out of 505 Skipped. AMG
Record 12 out of 505 Skipped. AFL
Record 13 out of 505 Skipped. A
Record 14 out of 505 Skipped. APD
Record 15 out of 505 Skipped. AKAM
Record 16 out of 505 Skipped. ALK
Record 17 out of 505 processed. ALB : SUCCESS(5034 rows)
Record 18 out of 505 processed. ARE : SUCCESS(5034 rows)
Record 19 out of 505 processed. ALXN : SUCCESS(5034 rows)
Record 20 out of 505 processed. ALGN : SUCCESS(4758 rows)
Record 21 out of 505 processed. ALLE : SUCCESS(1538 rows)
master_df saved, entering wait @ Sun Dec 29 11:10:41 2019 on item: AGN
Record 22 out of 505 processed. AGN : SUCCESS(5034 rows)
Record 2

In [2]:
'''
This iteration of the API call is a hybrid, spam/wait script. 

The API is spammed until the call limit reponse returns, then intitates a wait period to the next machine minute
(when the call limit resets).
'''



import time
import pandas as pd
from config import api_key_av
import requests

# creating limit response note from AV
bad_call = 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'

# creating loop counters & summary stat counters
success = 0 #success API call counter
failed = 0 #failed API call counter

now = int(time.time())
suffix = '1577635349'

# reading in query_list for required tickers
tickers = tickerlist

# creating master dataframe (to append to) and setting save path
save_path = f'../Resources/Master_Datasets/master_dataset_{suffix}.csv'

if suffix == now:
    master_df = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker'])
    master_df.to_csv(save_path, index=False)


# terminal print out of script performance
print(f'''
!Query Initiated!
-----------------
''')

# opening dataset to append to / read pre-appended items from
master_df = pd.read_csv(save_path)
ds_list = master_df['Ticker'].unique().tolist()


# primary loop for ticker call list to API
for record, ticker in enumerate(tickers):
    
    # by-passing previously called tickers
    if ticker not in ds_list:
        
        # setting API call parameters (ticker)
        params = {
            'function' : 'TIME_SERIES_DAILY',
            'symbol' : ticker,
            'outputsize' : 'full',
            'datatype' : 'json',
            'apikey' : api_key_av
        }
        
        # base API call URL
        url = 'https://www.alphavantage.co/query?'
        
        # t-e for handeling failed API calls
        try: 
            
            response = requests.get(url, params).json()
            
            # t-e for handeling call limit
            try:
                
                # checking if call limit reached
                if response['Note'] == bad_call:
                    
                    # dataframe save & wait timer for next machine minute
                    master_df.to_csv(save_path, index=False)
                    wait_time = time.asctime(time.localtime())
                    now = int(time.time())
                    print(f'master_df saved, entering wait @ {wait_time} on item: {ticker}')
                    
                    while now % 60 != 0:
                        now = int(time.time())
                    
                    wait_time = time.asctime(time.localtime())
                    print(f'Exiting wait @ {wait_time}')
                    
                    response = requests.get(url, params).json()
                    
                    # THE MEAT AND POTATOS
                    # creating main ticker dataframe
                    ticker_df = pd.DataFrame(response['Time Series (Daily)']).transpose().reset_index()
                    
                    # renaming columns to better indicators
                    ticker_df = ticker_df.rename(columns={
                        'index':'Date',
                        '1. open':'Open',
                        '2. high':'High',
                        '3. low':'Low',
                        '4. close':'Close',
                        '5. volume':'Volume'
                    })
                    
                    # pulling ticker symbol for confirmation purposes from API
                    ticker_df['Ticker'] = response['Meta Data']['2. Symbol']
                    
                    
                    # appending items to master dataframe
                    master_df = master_df.append(ticker_df)
                    
                    row_count = ticker_df['Date'].count()
                    
                    print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : SUCCESS({row_count} rows)')
                    
                    success += 1
            
            except:
                
                # THE MEAT AND POTATOS
                # creating main ticker dataframe
                ticker_df = pd.DataFrame(response['Time Series (Daily)']).transpose().reset_index()

                # renaming columns to better indicators
                ticker_df = ticker_df.rename(columns={
                    'index':'Date',
                    '1. open':'Open',
                    '2. high':'High',
                    '3. low':'Low',
                    '4. close':'Close',
                    '5. volume':'Volume'
                })
                
                # pulling ticker symbol for confirmation purposes from API
                ticker_df['Ticker'] = response['Meta Data']['2. Symbol']
                
                
                # appending items to master dataframe
                master_df = master_df.append(ticker_df)
                
                row_count = ticker_df['Date'].count()
                
                print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : SUCCESS({row_count} rows)')
                
                success += 1
                
        # handeling failed API calls
        except:
            print(f'Record {record + 1} out of {len(tickers)} processed. {ticker} : FAILED')
            
            failed += 1
            
    else:
        print(f'Record {record + 1} out of {len(tickers)} Skipped. {ticker}')
            
            

# writing master dataframe to csv   
master_df.to_csv(save_path, index=False)


# terminal print out of script performance
print(f'''
DONE
------------------
Query Summary
Success: {success} 
Failed: {failed}
Total: {record + 1}
''')


!Query Initiated!
-----------------

Record 1 out of 505 Skipped. MMM
Record 2 out of 505 Skipped. ABT
Record 3 out of 505 Skipped. ABBV
Record 4 out of 505 Skipped. ABMD
Record 5 out of 505 Skipped. ACN
Record 6 out of 505 Skipped. ATVI
Record 7 out of 505 Skipped. ADBE
Record 8 out of 505 Skipped. AMD
Record 9 out of 505 Skipped. AAP
Record 10 out of 505 Skipped. AES
Record 11 out of 505 Skipped. AMG
Record 12 out of 505 Skipped. AFL
Record 13 out of 505 Skipped. A
Record 14 out of 505 Skipped. APD
Record 15 out of 505 Skipped. AKAM
Record 16 out of 505 Skipped. ALK
Record 17 out of 505 Skipped. ALB
Record 18 out of 505 Skipped. ARE
Record 19 out of 505 Skipped. ALXN
Record 20 out of 505 Skipped. ALGN
Record 21 out of 505 Skipped. ALLE
Record 22 out of 505 Skipped. AGN
Record 23 out of 505 Skipped. ADS
Record 24 out of 505 Skipped. LNT
Record 25 out of 505 Skipped. ALL
Record 26 out of 505 Skipped. GOOGL
Record 27 out of 505 Skipped. GOOG
Record 28 out of 505 Skipped. MO
Record 29 o

Record 163 out of 505 processed. DXC : SUCCESS(732 rows)
master_df saved, entering wait @ Sun Dec 29 14:32:08 2019 on item: ETFC
Exiting wait @ Sun Dec 29 14:33:00 2019
Record 164 out of 505 processed. ETFC : SUCCESS(5034 rows)
Record 165 out of 505 processed. EMN : SUCCESS(5034 rows)
Record 166 out of 505 processed. ETN : SUCCESS(5034 rows)
Record 167 out of 505 processed. EBAY : SUCCESS(5034 rows)
Record 168 out of 505 processed. ECL : SUCCESS(5034 rows)
Record 169 out of 505 processed. EIX : SUCCESS(5034 rows)
master_df saved, entering wait @ Sun Dec 29 14:33:11 2019 on item: EW
Exiting wait @ Sun Dec 29 14:34:00 2019
Record 170 out of 505 processed. EW : SUCCESS(4971 rows)
Record 171 out of 505 processed. EA : SUCCESS(5034 rows)
Record 172 out of 505 processed. EMR : SUCCESS(5034 rows)
Record 173 out of 505 processed. ETR : SUCCESS(5034 rows)
Record 174 out of 505 processed. EOG : SUCCESS(5034 rows)
Record 175 out of 505 processed. EFX : SUCCESS(5034 rows)
master_df saved, entering

Record 270 out of 505 processed. JNJ : SUCCESS(5034 rows)
Record 271 out of 505 processed. JCI : SUCCESS(5034 rows)
master_df saved, entering wait @ Sun Dec 29 14:50:14 2019 on item: JPM
Exiting wait @ Sun Dec 29 14:51:00 2019
Record 272 out of 505 processed. JPM : SUCCESS(5034 rows)
Record 273 out of 505 processed. JNPR : SUCCESS(5034 rows)
Record 274 out of 505 processed. KSU : SUCCESS(5034 rows)
Record 275 out of 505 processed. K : SUCCESS(5034 rows)
Record 276 out of 505 processed. KEY : SUCCESS(5034 rows)
Record 277 out of 505 processed. KEYS : SUCCESS(1307 rows)
master_df saved, entering wait @ Sun Dec 29 14:51:12 2019 on item: KMB
Exiting wait @ Sun Dec 29 14:52:00 2019
Record 278 out of 505 processed. KMB : SUCCESS(5034 rows)
Record 279 out of 505 processed. KIM : SUCCESS(5031 rows)
Record 280 out of 505 processed. KMI : SUCCESS(2234 rows)
Record 281 out of 505 processed. KLAC : SUCCESS(5034 rows)
Record 282 out of 505 processed. KSS : SUCCESS(5034 rows)
Record 283 out of 505 p

Record 377 out of 505 processed. PSX : SUCCESS(1941 rows)
Record 378 out of 505 processed. PNW : SUCCESS(5034 rows)
Record 379 out of 505 processed. PXD : SUCCESS(5034 rows)
master_df saved, entering wait @ Sun Dec 29 15:08:13 2019 on item: PNC
Exiting wait @ Sun Dec 29 15:09:00 2019
Record 380 out of 505 processed. PNC : SUCCESS(5034 rows)
Record 381 out of 505 processed. PPG : SUCCESS(5034 rows)
Record 382 out of 505 processed. PPL : SUCCESS(5034 rows)
Record 383 out of 505 processed. PFG : SUCCESS(4577 rows)
Record 384 out of 505 processed. PG : SUCCESS(5034 rows)
Record 385 out of 505 processed. PGR : SUCCESS(5034 rows)
master_df saved, entering wait @ Sun Dec 29 15:09:15 2019 on item: PLD
Exiting wait @ Sun Dec 29 15:10:00 2019
Record 386 out of 505 processed. PLD : SUCCESS(5034 rows)
Record 387 out of 505 processed. PRU : SUCCESS(4541 rows)
Record 388 out of 505 processed. PEG : SUCCESS(5034 rows)
Record 389 out of 505 processed. PSA : SUCCESS(5034 rows)
Record 390 out of 505 pro

Record 484 out of 505 processed. WM : SUCCESS(5034 rows)
Record 485 out of 505 processed. WAT : SUCCESS(5034 rows)
Record 486 out of 505 processed. WEC : SUCCESS(5034 rows)
Record 487 out of 505 processed. WCG : SUCCESS(3901 rows)
master_df saved, entering wait @ Sun Dec 29 15:26:16 2019 on item: WFC
Exiting wait @ Sun Dec 29 15:27:00 2019
Record 488 out of 505 processed. WFC : SUCCESS(5034 rows)
Record 489 out of 505 processed. WELL : SUCCESS(4765 rows)
Record 490 out of 505 processed. WDC : SUCCESS(5034 rows)
Record 491 out of 505 processed. WU : SUCCESS(3333 rows)
Record 492 out of 505 processed. WRK : SUCCESS(1137 rows)
Record 493 out of 505 processed. WY : SUCCESS(5034 rows)
master_df saved, entering wait @ Sun Dec 29 15:27:15 2019 on item: WHR
Exiting wait @ Sun Dec 29 15:28:00 2019
Record 494 out of 505 processed. WHR : SUCCESS(5034 rows)
Record 495 out of 505 processed. WMB : SUCCESS(5034 rows)
Record 496 out of 505 processed. WLTW : SUCCESS(1003 rows)
Record 497 out of 505 pro